# Setup

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
import copy
from scipy.interpolate import CubicSpline
import torch.optim as optim
import torch.nn as nn
import torch
from torch.optim import Adam
from scipy.fftpack import fft, ifft
from scipy.stats import mode
from torch.utils.data import DataLoader, TensorDataset
import datetime
from sklearn.metrics import f1_score
import random

## Hyperparameters

In [2]:
num_epochs = 200
batch_size = 32  # Set your batch size
learning_rate_client = 0.001
local_epochs = 1
subject_dir = 'FL_Data/windowed_data_refused_4aug_UCI/subject_'  # Set your directory to the subject data
numclients = 75
num_classes = 8

#current timestamp
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
seed = 420
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Ensures that CUDA operations are deterministic
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load Data

In [4]:
def load_data_client(id, batch_size=batch_size, type='labelled_train'):
    # Load the data
    data = np.load(subject_dir + str(id) + '/windowed_' + type + '_x.npy')
    labels = np.load(subject_dir + str(id) + '/windowed_' + type + '_y.npy')

    # print shape of data
    # print(data.shape)
    # print(labels.shape)

    # Convert to torch tensor
    data = torch.from_numpy(data).float()
    labels = torch.from_numpy(labels).long()

    # Create a dataset
    dataset = torch.utils.data.TensorDataset(data, labels)

    # Create a dataloader
    if type == 'labelled_train' or type == 'unlabelled_train':
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    return dataloader

In [5]:
# for i in range(54):
#     data_label_train = load_data_client(i, batch_size, 'labelled_train')
#     data_unlabel_train = load_data_client(i, batch_size, 'unlabelled_train')
#     data_test = load_data_client(i, batch_size, 'test')

# Model Architecture

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Train and Test Function

In [7]:
# def train_autoencoder(model, train_loader, device, learning_rate=0.01, epochs=5):
#     model.to(device)
#     criterion = nn.MSELoss()
#     optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
#     model.train()
#     total_loss = 0
#     for epoch in range(epochs):
#         for data, target in train_loader:
#             data, target = data.to(device), target.to(device)
#             # print(data.shape)
#             data = data.permute(0, 2, 1)
#             optimizer.zero_grad()
#             output = model(data)
#             # print(output.shape)
#             loss = criterion(output, data)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
        
#         epoch_loss = total_loss / len(train_loader)
#         # print(f'Epoch {epoch+1}, Loss: {epoch_loss}')
#         total_loss = 0  # Reset total loss for the next epoch

#     results = {
#         'train_loss': epoch_loss
#     }
    
#     return results  # Returns the average loss of the last epoch

In [8]:
# def test_autoencoder(model, test_loader, device):
#     model.to(device)
#     model.eval()
    
#     criterion = nn.MSELoss()
#     total_loss = 0
    
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             data = data.permute(0, 2, 1)
#             output = model(data)
#             loss = criterion(output, data)
#             total_loss += loss.item()
    
#     avg_loss = total_loss / len(test_loader)
#     # print(f'Test Loss: {avg_loss}')
    
#     return avg_loss  # Returns the average loss for the test data

In [9]:
# function to train the model
def train_model(model, train_loader, device, learning_rate=0.001, epochs=1):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    model.train()
    total_loss = 0
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        epoch_loss = total_loss / len(train_loader)
        total_loss = 0  # Reset total loss for the next epoch

    results = {
        'train_loss': epoch_loss
    }
    
    return results  # Returns the average loss of the last epoch

In [10]:
# function to test the model
# method to test the model and get the accuracy and f1 score
def test_model(model, test_loader):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')
    # print(f'Accuracy: {accuracy}, F1 Score: {f1}')
    results = {
        'accuracy': accuracy,
        'f1': f1
    }

    return results

# Client

In [11]:
class Client():
  def __init__(self, client_config:dict):
    # client config as dict to make configuration dynamic
    self.id = client_config["id"]
    self.config = client_config
    self.__model = None

    self.labelled_loader = self.config["labelled"]
    self.unlabelled_loader = self.config["unlabelled"]
    self.test_loader = self.config["test"]

  @property
  def model(self):
    return self.__model

  @model.setter
  def model(self, model):
    self.__model = model

  def __len__(self):
    """Return a total size of the client's local data."""
    return len(self.unlabelled_loader.sampler)

  def train_ssl(self):
    results = train_model(model = self.model,
                    train_loader = self.unlabelled_loader,
                    device=device,
                    learning_rate=learning_rate_client,
                    epochs=local_epochs)
    print(f"Train result client {self.id}: {results}")

  def test_ssl(self):
    loss = test_model(model = self.model,
                    test_loader = self.unlabelled_loader)
    print(f"Test result client {self.id}: {loss}")
    return loss

# Server

In [12]:
class FedAvg():
  def __init__(self):
    self.globalmodel = CNNFeatureExtractor(num_classes=5) # number for augmentations
    self.rounds = 0
    self.params = {}

  def aggregate(self, round):
    #v1:update the aggregate to save the model with round and date indicator
    modelparams = []
    for i in self.params.keys():
      modelparams.append(self.params[i])

    avg_weights = {}
    for name in modelparams[0].keys():
      avg_weights[name] = torch.mean(torch.stack([w[name] for w in modelparams]), dim = 0)

    self.globalmodel.load_state_dict(avg_weights)

    #save the model
    # name_path = f'Refused_FL/Model_Global/{current_time}' #v1: only 4 augmentation
    name_path = f'Refused_FL/Model_Global_4Aug_UCI/{current_time}'
    if not os.path.exists(name_path):
      os.makedirs(name_path)

    torch.save(self.globalmodel.state_dict(), f"{name_path}/global_model_round_{round}.pth")
    
    # filename = f"{path_glob_m}/global_model_round_{round}_{current_time}.pth"
    # torch.save(self.globalmodel.state_dict(), filename)

  def clientstrain(self, clientconfig):
    clients = clientconfig
    for i in clients.keys():
      test_client = Client(clients[i])
      test_client.model = copy.deepcopy(self.globalmodel)
      test_client.train_ssl()
      test_client.test_ssl()
      self.params[i] = test_client.model.state_dict()

  def initiate_FL(self, clientconfig, serverdata):
    clients = clientconfig
    print("Round: {}".format(self.rounds))

    print("Obtaining Weights!!")
    self.clientstrain(clients)

    #### Aggregate model
    print("Aggregating Model!!")
    self.aggregate(self.rounds)

    #### Replace parameters with global model parameters
    for i in self.params.keys():
        self.params[i] = self.globalmodel.state_dict()


    servertest = serverdata
    result = test_model(model = self.globalmodel,
                    test_loader = servertest)
    print("Round {} metrics:".format(self.rounds))
    print("Server Result = {}".format(result))
    print("Round {} finished!".format(self.rounds))
    self.rounds += 1
    return clients, result

# Main

In [13]:
clients = {}

for i in range(numclients):
    clients[i] = {"id": i, "batch_size": batch_size, "local_epoch": 1}
    clients[i]['labelled'] = load_data_client(i, batch_size, 'labelled_train')
    clients[i]['unlabelled'] = load_data_client(i, batch_size, 'unlabelled_train')
    clients[i]['test'] = load_data_client(i, batch_size, 'test')

    print(f"client: {i}")
    print(f"labelled: {len(clients[i]['labelled'])}")
    print(f"unlabelled: {len(clients[i]['unlabelled'])}")
    print(f"test: {len(clients[i]['test'])}")

# combine all client test data into one
combined_test_data = []
combined_test_labels = []
for i in range(numclients):
    for data, labels in clients[i]['test']:
        combined_test_data.append(data)
        combined_test_labels.append(labels)
combined_test_data = torch.cat(combined_test_data, dim=0)
combined_test_labels = torch.cat(combined_test_labels, dim=0)
# create dataset and dataloader
combined_test_dataset = torch.utils.data.TensorDataset(combined_test_data, combined_test_labels)
combined_test_dataloader = torch.utils.data.DataLoader(combined_test_dataset, batch_size=batch_size, shuffle=False)

print(f"combined test: {len(combined_test_dataloader)}")

# combine all client unlabelled data into one
combined_unlabelled_data = []
combined_unlabelled_labels = []
for i in range(numclients):
    for data, labels in clients[i]['unlabelled']:
        combined_unlabelled_data.append(data)
        combined_unlabelled_labels.append(labels)
combined_unlabelled_data = torch.cat(combined_unlabelled_data, dim=0)
combined_unlabelled_labels = torch.cat(combined_unlabelled_labels, dim=0)
# create dataset and dataloader
combined_unlabelled_dataset = torch.utils.data.TensorDataset(combined_unlabelled_data, combined_unlabelled_labels)
combined_unlabelled_dataloader = torch.utils.data.DataLoader(combined_unlabelled_dataset, batch_size=batch_size, shuffle=True)

print(f"combined unlabelled: {len(combined_unlabelled_dataloader)}")

# server test_data
server_test_data = combined_unlabelled_dataloader

start = time.time()
server = FedAvg()

loss_rounds = []
for i in range(num_epochs):
    clients, loss = server.initiate_FL(clientconfig=clients, serverdata=server_test_data)
    loss_rounds.append(loss)

print("\n")
print("-" * 50)
print("Loss values all rounds: ", loss_rounds)
print(f"\nTime cost: {round((time.time()-start)/60, 2)}min.")

client: 0
labelled: 2
unlabelled: 6
test: 2
client: 1
labelled: 2
unlabelled: 5
test: 2
client: 2
labelled: 2
unlabelled: 5
test: 2
client: 3
labelled: 2
unlabelled: 5
test: 2
client: 4
labelled: 2
unlabelled: 5
test: 2
client: 5
labelled: 2
unlabelled: 5
test: 2
client: 6
labelled: 2
unlabelled: 5
test: 2
client: 7
labelled: 2
unlabelled: 5
test: 2
client: 8
labelled: 2
unlabelled: 5
test: 2
client: 9
labelled: 2
unlabelled: 5
test: 2
client: 10
labelled: 2
unlabelled: 5
test: 2
client: 11
labelled: 2
unlabelled: 5
test: 2
client: 12
labelled: 2
unlabelled: 5
test: 2
client: 13
labelled: 2
unlabelled: 5
test: 2
client: 14
labelled: 2
unlabelled: 5
test: 2
client: 15
labelled: 2
unlabelled: 6
test: 2
client: 16
labelled: 2
unlabelled: 6
test: 2
client: 17
labelled: 2
unlabelled: 6
test: 2
client: 18
labelled: 2
unlabelled: 6
test: 2
client: 19
labelled: 2
unlabelled: 6
test: 2
client: 20
labelled: 2
unlabelled: 6
test: 2
client: 21
labelled: 2
unlabelled: 5
test: 2
client: 22
labelled:

combined test: 328
combined unlabelled: 1053
Round: 0
Obtaining Weights!!
Train result client 0: {'train_loss': 1.5993586977322896}
Test result client 0: {'accuracy': 0.2716049382716049, 'f1': 0.11602541052379242}
Train result client 1: {'train_loss': 1.6028224229812622}
Test result client 1: {'accuracy': 0.2535211267605634, 'f1': 0.10254787149865484}
Train result client 2: {'train_loss': 1.5984936952590942}
Test result client 2: {'accuracy': 0.2830188679245283, 'f1': 0.12486126526082131}
Train result client 3: {'train_loss': 1.5982758760452271}
Test result client 3: {'accuracy': 0.3013698630136986, 'f1': 0.13958183129055515}
Train result client 4: {'train_loss': 1.6059281826019287}
Test result client 4: {'accuracy': 0.20567375886524822, 'f1': 0.07017104714226116}
Train result client 5: {'train_loss': 1.5966423511505128}
Test result client 5: {'accuracy': 0.28859060402684567, 'f1': 0.12926454138702465}
Train result client 6: {'train_loss': 1.5930160284042358}
Test result client 6: {'ac

# Fine-Tuned Phase

## Model Fine-tuned

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Data Fine-Tuned

In [15]:
# combine all client labelled data into one
combined_labelled_data = []
combined_labelled_labels = []
for i in range(numclients):
    for data, labels in clients[i]['labelled']:
        combined_labelled_data.append(data)
        combined_labelled_labels.append(labels)
combined_labelled_data = torch.cat(combined_labelled_data, dim=0)
combined_labelled_labels = torch.cat(combined_labelled_labels, dim=0)
# create dataset and dataloader
combined_labelled_dataset = torch.utils.data.TensorDataset(combined_labelled_data, combined_labelled_labels)
combined_labelled_dataloader = torch.utils.data.DataLoader(combined_labelled_dataset, batch_size=batch_size, shuffle=True)

print(f"combined labelled: {len(combined_labelled_dataloader)}")

combined labelled: 263


### Class Weight

In [16]:
# Count the frequency of each class
class_counts = torch.zeros(num_classes)  # num_classes should be defined based on your dataset
for _, target in combined_labelled_dataloader:
    class_counts += torch.bincount(target, minlength=num_classes)

# Calculate class weights
class_counts += 1  # Add 1 to each class count to avoid division by zero
c_weight = 1. / class_counts
c_weight = c_weight / c_weight.sum() * num_classes
c_weight = c_weight.to(device)

In [17]:
class_counts

tensor([1.7700e+03, 1.3930e+03, 1.5610e+03, 1.2540e+03, 2.7100e+02, 8.0600e+02,
        1.3400e+03, 1.0000e+00])

In [18]:
c_weight

tensor([4.4821e-03, 5.6952e-03, 5.0822e-03, 6.3265e-03, 2.9274e-02, 9.8429e-03,
        5.9204e-03, 7.9334e+00], device='cuda:0')

## Model

In [19]:
# pretrained_model_path = 'Refused_FL/Model_Global/2024-02-11_22-01-36/global_model_round_199.pth' # model with 200 epochs, 1 local epoch
# model = CNNFeatureExtractor(num_classes=4)
pretrained_model_path = f'Refused_FL/Model_Global_4Aug_UCI/{current_time}/global_model_round_49.pth' # model with 200 epochs, 1 local epoch
model = CNNFeatureExtractor(num_classes=5)

#load pretrained model
model.load_state_dict(torch.load(pretrained_model_path))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

CNNFeatureExtractor(
  (conv1): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=8, bias=True)
)

In [20]:
# model = CNNFeatureExtractor(num_classes=num_classes)
# model.to(device)

## Fine-Tuning

In [21]:
# method to test the model and get the accuracy and f1 score
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}, F1 Score: {f1}')
    return accuracy, f1

In [22]:
def fine_tune_model(model, train_loader, test_loader, num_epochs=200):
    # Assuming class weights are calculated and provided as `class_weights`
    class_weights = torch.tensor(c_weight).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)
    
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            data = data.permute(0, 2, 1)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        acc, f1 = test_model(model, test_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {acc}, F1 Score: {f1}')

In [23]:
fine_tune_model(model.to(device), combined_labelled_dataloader, combined_test_dataloader,num_epochs=50)

C:\Users\IME-LAB\AppData\Local\Temp\ipykernel_14540\3333203605.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights = torch.tensor(c_weight).to(device)


Accuracy: 0.30071462601238685, F1 Score: 0.18594347241006068
Epoch 1/50, Loss: 1.787535309791565, Accuracy: 0.30071462601238685, F1 Score: 0.18594347241006068
Accuracy: 0.3298713673177704, F1 Score: 0.24646951952576898
Epoch 2/50, Loss: 1.7320255041122437, Accuracy: 0.3298713673177704, F1 Score: 0.24646951952576898
Accuracy: 0.3295855169128156, F1 Score: 0.2449180503875897
Epoch 3/50, Loss: 1.6418414115905762, Accuracy: 0.3295855169128156, F1 Score: 0.2449180503875897
Accuracy: 0.3156741305383516, F1 Score: 0.24050821679564602
Epoch 4/50, Loss: 1.7756448984146118, Accuracy: 0.3156741305383516, F1 Score: 0.24050821679564602
Accuracy: 0.3301572177227251, F1 Score: 0.2542401303060689
Epoch 5/50, Loss: 1.1774240732192993, Accuracy: 0.3301572177227251, F1 Score: 0.2542401303060689
Accuracy: 0.3250119104335398, F1 Score: 0.26191388751398825
Epoch 6/50, Loss: 1.3704676628112793, Accuracy: 0.3250119104335398, F1 Score: 0.26191388751398825
Accuracy: 0.33758932825154836, F1 Score: 0.262777458057

In [24]:
test_model(model, combined_test_dataloader)

Accuracy: 0.348261076703192, F1 Score: 0.34099234158792485


(0.348261076703192, 0.34099234158792485)